### below load dependency pkgs. and then create manually a new directory under experiments like base_model having params.json. this json file contains hyperparameter related to model, user can tweak these parameters for finding best model. and based on new name , change below model_dir path.

In [1]:
import argparse
import logging
import os

import numpy as np
import torch
from torch.autograd import Variable
import torch.optim as optim
from tqdm import trange
import random

# dependancy packages.
import utils
import model.net as net
from model.data_loader import DataLoader


data_dir='data/small'
model_dir='experiments/base_model'
word2vec_dir='experiments/word2vec' # keep GoogleNews-vectors-negative300.bin inside this directory.

# ## Let's use CoNLL 2002 data to build a NER system
# 
# CoNLL2002 corpus is available in NLTK. We use Spanish data.

In [2]:
import nltk
from sklearn.model_selection import cross_val_predict
nltk.corpus.conll2002.fileids()

train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
train_sents = [val for val in train_sents if len(val)!=0]
train_sents = train_sents[:5000]
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
test_sents = [val for val in test_sents if len(val)!=0]
test_sents = test_sents[:1000]

print(len(train_sents))
print(len(test_sents))


5000
1000


In [3]:
tokens_word2vec_list = []
tr_tokens_list = []
tr_pos_list = []
tr_labels_list = []
ts_tokens_list = []
ts_pos_list = []
ts_labels_list = []

for sents in train_sents:
    tokens = tuple(i[0].lower() for i in sents)
    pos_tags = [i[1] for i in sents]
    labels = tuple(i[2] for i in sents)
    tokens_word2vec_list.append(tokens)
    tr_tokens_list.append(tokens)
    tr_pos_list.append(pos_tags)
    tr_labels_list.append(labels)

for sents in test_sents:
    tokens = tuple(i[0].lower() for i in sents)
    pos_tags = [i[1] for i in sents]
    labels = tuple(i[2] for i in sents)
    tokens_word2vec_list.append(tokens)
    ts_tokens_list.append(tokens)
    ts_pos_list.append(pos_tags)
    ts_labels_list.append(labels)

print(tr_tokens_list[0])
print(tr_pos_list[:4])
print(tr_labels_list[0])

('melbourne', '(', 'australia', ')', ',', '25', 'may', '(', 'efe', ')', '.')
[['NP', 'Fpa', 'NP', 'Fpt', 'Fc', 'Z', 'NC', 'Fpa', 'NC', 'Fpt', 'Fp'], ['Fg'], ['DA', 'NC', 'AQ', 'SP', 'NC', 'Fc', 'VMI', 'NC', 'Fc', 'VMI', 'RG', 'DA', 'NC', 'SP', 'VMN', 'NC', 'SP', 'VMN', 'SP', 'NC', 'AQ', 'AQ', 'RG', 'SP', 'DI', 'NC', 'SP', 'NC', 'PR', 'VMI', 'DA', 'NC', 'SP', 'DA', 'NC', 'AQ', 'SP', 'DA', 'NC', 'Fp'], ['DA', 'NC', 'SP', 'NC', 'AQ', 'VMI', 'NC', 'RG', 'SP', 'CS', 'DI', 'NC', 'SP', 'NC', 'AQ', 'SP', 'NC', 'SP', 'NC', 'Fpa', 'NP', 'Fpt', 'P0', 'VMS', 'AQ', 'SP', 'VMN', 'DI', 'NC', 'AQ', 'CC', 'VMN', 'DA', 'NC', 'SP', 'DA', 'NC', 'SP', 'DA', 'NC', 'SP', 'CS', 'DA', 'NC', 'PR', 'PP', 'VMI', 'VMI', 'VAN', 'VMP', 'NC', 'SP', 'DA', 'VMP', 'SP', 'NC', 'SP', 'DA', 'NC', 'AQ', 'Fp']]
('B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O')


In [4]:
from gensim.models import Word2Vec
sentences = tokens_word2vec_list#[["cat", "say", "meow"], ["dog", "say", "woof"]]

model = Word2Vec(min_count=1)
model.build_vocab(sentences)  # prepare the model vocabulary
model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)  # train word vectors

/home/adzuser/user_achyuta/venv3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


(637031, 960020)

In [5]:
model['australia'].astype(np.float64)

/home/adzuser/user_achyuta/venv3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.03542154, -0.01396509, -0.0059225 , -0.0747686 ,  0.03587869,
        0.05271797,  0.15769364,  0.05751861,  0.01524669, -0.00404117,
       -0.02365159, -0.04342028,  0.05798311,  0.03871023,  0.02839377,
       -0.02556238,  0.11380405,  0.06572264, -0.04511902,  0.00613007,
       -0.00046499, -0.03643322,  0.07480255,  0.01360533,  0.0403607 ,
        0.04769563,  0.10762448,  0.02940772, -0.08135953,  0.09551089,
       -0.10398597, -0.09710502,  0.02616859, -0.05593977, -0.04623265,
        0.07723986,  0.01155668,  0.03138071,  0.0359161 ,  0.0899602 ,
       -0.06940252, -0.07332759, -0.0480862 , -0.02716609,  0.09468073,
       -0.03155367,  0.10605608,  0.04205232, -0.01401343,  0.02569293,
        0.06696339, -0.03269048,  0.03017501,  0.01148963, -0.02771042,
        0.04406427,  0.10084617,  0.03856216,  0.00645268,  0.03385818,
       -0.05993786,  0.04428807, -0.05095945, -0.03825443,  0.02445826,
        0.07199966,  0.03509669, -0.03542428, -0.14442119,  0.01

In [6]:
"""Build vocabularies of words and tags from datasets"""

import argparse
from collections import Counter
import json
import os


parser = {'min_count_word':1,'min_count_tag':1,'data_dir':'data/small'}#argparse.ArgumentParser()
#parser.add_argument('--min_count_word', default=1, help="Minimum count for words in the dataset", type=int)
#parser.add_argument('--min_count_tag', default=1, help="Minimum count for tags in the dataset", type=int)
#parser.add_argument('--data_dir', default='data/small', help="Directory containing the dataset")

# Hyper parameters for the vocab
PAD_WORD = '<pad>'
PAD_TAG = 'O'
UNK_WORD = 'UNK'


def save_vocab_to_txt_file(vocab, txt_path):
    """Writes one token per line, 0-based line id corresponds to the id of the token.

    Args:
        vocab: (iterable object) yields token
        txt_path: (stirng) path to vocab file
    """
    with open(txt_path, "w") as f:
        for token in vocab:
            f.write(token + '\n')
            

def save_dict_to_json(d, json_path):
    """Saves dict to json file

    Args:
        d: (dict)
        json_path: (string) path to json file
    """
    with open(json_path, 'w') as f:
        d = {k: v for k, v in d.items()}
        json.dump(d, f, indent=4)


def update_vocab(tokens_list, vocab):
    """Update word and tag vocabulary from dataset

    Args:
        txt_path: (string) path to file, one sentence per line
        vocab: (dict or Counter) with update method

    Returns:
        dataset_size: (int) number of elements in the dataset
    """
    for i, tokens in enumerate(tokens_list):
        vocab.update(tokens)

    return i + 1


if __name__ == '__main__':
    #args = parser.parse_args()

    # Build word vocab with train and test datasets
    print("Building word vocabulary...")
    words = Counter()
    size_train_sentences = update_vocab(tr_tokens_list, words)
    size_dev_sentences = update_vocab(ts_tokens_list, words)
    size_test_sentences = update_vocab(ts_tokens_list, words)
    print("- done.")

    # Build tag vocab with train and test datasets
    print("Building tag vocabulary...")
    tags = Counter()
    size_train_tags = update_vocab(tr_labels_list, tags)
    size_dev_tags = update_vocab(ts_labels_list, tags)
    size_test_tags = update_vocab(ts_labels_list, tags)
    print("- done.")

    # Assert same number of examples in datasets
    assert size_train_sentences == size_train_tags
    assert size_dev_sentences == size_dev_tags
    assert size_test_sentences == size_test_tags

    # Only keep most frequent tokens
    words = [tok for tok, count in words.items() if count >= parser['min_count_word']]
    tags = [tok for tok, count in tags.items() if count >= parser['min_count_tag']]

    # Add pad tokens
    if PAD_WORD not in words: words.append(PAD_WORD)
    if PAD_TAG not in tags: tags.append(PAD_TAG)
    
    # add word for unknown words 
    words.append(UNK_WORD)

    # Save vocabularies to file
    print("Saving vocabularies to file...")
    save_vocab_to_txt_file(words, os.path.join(parser['data_dir'], 'words.txt'))
    save_vocab_to_txt_file(tags, os.path.join(parser['data_dir'], 'tags.txt'))
    print("- done.")

    # Save datasets properties in json file
    sizes = {
        'train_size': size_train_sentences,
        'dev_size': size_dev_sentences,
        'test_size': size_test_sentences,
        'vocab_size': len(words),
        'number_of_tags': len(tags),
        'pad_word': PAD_WORD,
        'pad_tag': PAD_TAG,
        'unk_word': UNK_WORD
    }
    
    save_dict_to_json(sizes, os.path.join(parser['data_dir'], 'dataset_params.json'))

    # Logging sizes
    to_print = "\n".join("- {}: {}".format(k, v) for k, v in sizes.items())
    print("Characteristics of the dataset:\n{}".format(to_print))

Building word vocabulary...
- done.
Building tag vocabulary...
- done.
Saving vocabularies to file...
- done.
Characteristics of the dataset:
- train_size: 5000
- dev_size: 1000
- test_size: 1000
- vocab_size: 19952
- number_of_tags: 9
- pad_word: <pad>
- pad_tag: O
- unk_word: UNK


In [7]:
'''
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(pos_list)#([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])   
print(enc.n_values_)
print(enc.feature_indices_)
print(enc.transform([('NP', 'Fpa', 'NP', 'Fpt', 'Fc', 'Z', 'NC', 'Fpa', 'NC', 'Fpt', 'Fp')]).toarray())
'''
from numpy import argmax
import itertools

# define input string
data = ['NP', 'Fpa', 'NP', 'Fpt', 'Fc', 'Z', 'NC', 'Fpa', 'NC', 'Fpt', 'Fp']

# define universe of possible input values
unq_pos_list = list(itertools.chain.from_iterable(tr_pos_list+ts_pos_list))
unq_pos_list = list(set(unq_pos_list))
print(unq_pos_list)

# define a mapping of chars to integers
char_to_int = dict((c, i) for i, c in enumerate(unq_pos_list))
int_to_char = dict((i, c) for i, c in enumerate(unq_pos_list))

def onehot_encode(data,unq_pos_list,char_to_int):
    # integer encode input data
    integer_encoded = [char_to_int[pos] for pos in data]
    print(integer_encoded)

    # one hot encode
    onehot_encoded = list()
    for value in integer_encoded:
	    letter = [0 for _ in range(len(unq_pos_list))]
	    letter[value] = 1
	    onehot_encoded.append(letter)
    #print(onehot_encoded)
    return onehot_encoded

onehot_encoded = onehot_encode(data,unq_pos_list,char_to_int)
print(onehot_encoded)

# invert encoding
inverted = int_to_char[argmax(onehot_encoded[0])]
print(inverted)

['Y', 'DN', 'Fs', 'VSN', 'Fc', 'P0', 'Fit', 'DT', 'Fpt', 'VMS', 'CC', 'PN', 'VSG', 'DP', 'Faa', 'VMM', 'Fg', 'Fh', 'SP', 'Fe', 'Z', 'NC', 'Fd', 'VSP', 'DI', 'PP', 'PT', 'VAS', 'AO', 'Fp', 'Ft', 'DA', 'PR', 'VSS', 'Fat', 'VMP', 'Fia', 'AQ', 'VSI', 'PX', 'PD', 'VAP', 'VSM', 'RG', 'I', 'Fpa', 'VAI', 'VMN', 'VAN', 'VMI', 'NP', 'CS', 'Fz', 'PI', 'RN', 'VMG', 'Fx', 'DD']
[50, 45, 50, 8, 4, 20, 21, 45, 21, 8, 29]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# train & evaluation wrappers.

In [8]:
class data_loader(object):
    def data_iterator(self, data_dir, data, params, shuffle=False):
        """
        Returns a generator that yields batches data with labels. Batch size is params.batch_size. Expires after one
        pass over the data.

        Args:
            data: (dict) contains data which has keys 'data', 'labels' and 'size'
            params: (Params) hyperparameters of the training process.
            shuffle: (bool) whether the data should be shuffled

        Yields:
            batch_data: (Variable) dimension batch_size x seq_len with the sentence data
            batch_labels: (Variable) dimension batch_size x seq_len with the corresponding labels

        """

        # make a list that decides the order in which we go over the data- this avoids explicit shuffling of data
        order = list(range(data['size']))
        if shuffle:
            random.seed(230)
            random.shuffle(order)

        # loading dataset_params
        json_path = os.path.join(data_dir, 'dataset_params.json')
        assert os.path.isfile(json_path), "No json file found at {}, run build_vocab.py".format(json_path)
        dataset_params = utils.Params(json_path)        

        # loading vocab (we require this to map words to their indices)
        vocab_path = os.path.join(data_dir, 'words.txt')
        vocab = {}
        with open(vocab_path) as f:
            for i, l in enumerate(f.read().splitlines()):
                vocab[l] = i

        # setting the indices for UNKnown words and PADding symbols
        self.unk_ind = vocab[dataset_params.unk_word]
        self.pad_ind = vocab[dataset_params.pad_word]
        

        # one pass over data
        for i in range((data['size']+1)//params.batch_size):
            # fetch sentences and tags
            batch_sentences = [data['data'][idx] for idx in order[i*params.batch_size:(i+1)*params.batch_size]]
            batch_tags = [data['labels'][idx] for idx in order[i*params.batch_size:(i+1)*params.batch_size]]

            # compute length of longest sentence in batch
            batch_max_len = max([len(s) for s in batch_sentences])

            # prepare a numpy array with the data, initialising the data with pad_ind and all labels with -1
            # initialising labels to -1 differentiates tokens with tags from PADding tokens
            batch_data = self.pad_ind*np.ones((len(batch_sentences), batch_max_len, 100))
            batch_labels = -1*np.ones((len(batch_sentences), batch_max_len))
            
            print("------------------------------")
            print(np.array(batch_sentences).shape)
            print(batch_data.shape)
            print(batch_labels.shape)

            # copy the data to the numpy array
            for j in range(len(batch_sentences)):
                cur_len = len(batch_sentences[j])
                print(cur_len)
                print(batch_data[j][:cur_len].shape)
                batch_data[j][:cur_len] = batch_sentences[j]
                batch_labels[j][:cur_len] = batch_tags[j]

            # since all data are indices, we convert them to torch LongTensors
            batch_data, batch_labels = torch.LongTensor(batch_data), torch.LongTensor(batch_labels)
            print(type(batch_data))
            print(type(batch_labels))
            batch_data = batch_data.to(dtype=torch.long)
            batch_labels = batch_labels.to(dtype=torch.long)

            # shift tensors to GPU if available
            if params.cuda:
                batch_data, batch_labels = batch_data.cuda(), batch_labels.cuda()

            # convert them to Variables to record operations in the computational graph
            batch_data, batch_labels = Variable(batch_data), Variable(batch_labels)
            print(type(batch_data))
            print(type(batch_labels))
            print(batch_data.dtype)
            print(batch_data[0].dtype)
            print(batch_data[0])

            yield batch_data, batch_labels


def train(model, optimizer, loss_fn, data_iterator, metrics, params, num_steps):
    """Train the model on `num_steps` batches

    Args:
        model: (torch.nn.Module) the neural network
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        data_iterator: (generator) a generator that generates batches of data and labels
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to training mode
    model.train()

    # summary for current training loop and a running average object for loss
    summ = []
    loss_avg = utils.RunningAverage()
    
    # Use tqdm for progress bar
    t = trange(num_steps) 
    for i in t:
        # fetch the next training batch
        train_batch, labels_batch = next(data_iterator)
        print("-------------------------")
        print(train_batch.shape)
        print(labels_batch.shape)

        # compute model output and loss
        output_batch = model(train_batch)
        loss = loss_fn(output_batch, labels_batch)

        # clear previous gradients, compute gradients of all variables wrt loss
        optimizer.zero_grad()
        loss.backward()

        # performs updates using calculated gradients
        optimizer.step()

        # Evaluate summaries only once in a while
        if i % params.save_summary_steps == 0:
            # extract data from torch Variable, move to cpu, convert to numpy arrays
            output_batch = output_batch.data.cpu().numpy()
            labels_batch = labels_batch.data.cpu().numpy()

            # compute all metrics on this batch
            summary_batch = {metric:metrics[metric](output_batch, labels_batch)
                             for metric in metrics}
            summary_batch['loss'] = loss.data.item()
            summ.append(summary_batch)

        # update the average loss
        loss_avg.update(loss.data.item())
        t.set_postfix(loss='{:05.3f}'.format(loss_avg()))

    # compute mean of all metrics in summary
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]} 
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())
    

def evaluate(model, loss_fn, data_iterator, metrics, params, num_steps):
    """Evaluate the model on `num_steps` batches.

    Args:
        model: (torch.nn.Module) the neural network
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        data_iterator: (generator) a generator that generates batches of data and labels
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to evaluation mode
    model.eval()

    # summary for current eval loop
    summ = []

    # compute metrics over the dataset
    for _ in range(num_steps):
        # fetch the next evaluation batch
        data_batch, labels_batch = next(data_iterator)
        
        # compute model output
        output_batch = model(data_batch)
        loss = loss_fn(output_batch, labels_batch)

        # extract data from torch Variable, move to cpu, convert to numpy arrays
        output_batch = output_batch.data.cpu().numpy()
        labels_batch = labels_batch.data.cpu().numpy()

        # compute all metrics on this batch
        summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                         for metric in metrics}
        summary_batch['loss'] = loss.data.item()
        summ.append(summary_batch)

    # compute mean of all metrics in summary
    metrics_mean = {metric:np.mean([x[metric] for x in summ]) for metric in summ[0]} 
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_mean.items())
    return metrics_mean

def train_and_evaluate(model, train_data, val_data, optimizer, loss_fn, metrics, params, model_dir, data_dir, restore_file=None):
    """Train the model and evaluate every epoch.

    Args:
        model: (torch.nn.Module) the neural network
        train_data: (dict) training data with keys 'data' and 'labels'
        val_data: (dict) validaion data with keys 'data' and 'labels'
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        model_dir: (string) directory containing config, weights and log
        restore_file: (string) optional- name of file to restore from (without its extension .pth.tar)
    """
    # reload weights from restore_file if specified
    if restore_file is not None:
        restore_path = os.path.join(args.model_dir, args.restore_file + '.pth.tar')
        utils.load_checkpoint(restore_path, model, optimizer)
        
    best_val_acc = 0.0

    for epoch in range(params.num_epochs):
        # Run one epoch
        # compute number of batches in one epoch (one full pass over the training set)
        num_steps = (params.train_size + 1) // params.batch_size
        dl = data_loader()
        train_data_iterator = dl.data_iterator(data_dir, train_data, params, shuffle=True)
        train(model, optimizer, loss_fn, train_data_iterator, metrics, params, num_steps)
            
        # Evaluate for one epoch on validation set
        num_steps = (params.val_size + 1) // params.batch_size
        val_data_iterator = data_iterator(val_data, params, shuffle=False)
        val_metrics = evaluate(model, loss_fn, val_data_iterator, metrics, params, num_steps)
        
        val_acc = val_metrics['accuracy']
        is_best = val_acc >= best_val_acc

        # Save weights
        utils.save_checkpoint({'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optim_dict' : optimizer.state_dict()}, 
                               is_best=is_best,
                               checkpoint=model_dir)
            
        # If best_eval, best_save_path        
        if is_best:
            best_val_acc = val_acc
            
            # Save best val metrics in a json file in the model directory
            best_json_path = os.path.join(model_dir, "metrics_val_best_weights.json")
            utils.save_dict_to_json(val_metrics, best_json_path)

        # Save latest val metrics in a json file in the model directory
        last_json_path = os.path.join(model_dir, "metrics_val_last_weights.json")
        utils.save_dict_to_json(val_metrics, last_json_path)

# Train

In [9]:
restore_file=None

# Load the parameters from json file
json_path = os.path.join(model_dir, 'params.json')
assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
params = utils.Params(json_path)

# use GPU if available
params.cuda = torch.cuda.is_available()
    
# Set the random seed for reproducible experiments
torch.manual_seed(230)
if params.cuda: torch.cuda.manual_seed(230)
        
# Set the logger
utils.set_logger(os.path.join(model_dir, 'train.log'))

# Create the input data pipeline
logging.info("Loading the datasets...")
    
# load data
#data_loader = DataLoader(data_dir, params)
#data = data_loader.load_data(['train', 'val'], data_dir)
#train_data = data['train']
#val_data = data['val']
params.vocab_size=sizes['vocab_size']
params.number_of_tags = sizes['number_of_tags']

# add word2vec
'''
train_data = {'data':[[],[]],'labels':[[]],'size':10}
'''
tr_data = None
tr_pos = None
ts_data = None
ts_pos = None
def preprocess_data(tokens_list,pos_list):
    tr_data = []
    for v in tokens_list[:5]:
        tmp_tr_data = []
        for v_in in v:
            tmp_tr_data.append(model[v_in.lower()].tolist()) #.astype(np.float64)
        tr_data.append(tmp_tr_data)
    
    tr_pos = [onehot_encode(v,unq_pos_list,char_to_int) for v in pos_list[:5]]
    return tr_data,tr_pos

tr_data,tr_pos = preprocess_data(tr_tokens_list,tr_pos_list)
ts_data,ts_pos = preprocess_data(ts_tokens_list,ts_pos_list)


# loading tags (we require this to map tags to their indices)
tags_path = os.path.join(data_dir, 'tags.txt')
tag_map = {}
with open(tags_path) as f:
    for i, t in enumerate(f.read().splitlines()):
        tag_map[t] = i

def preprocess_label(tag_map, labels_list):
    fnl_labels = []
    for label_sent in tr_labels_list:
        tmp_label = []
        for label in label_sent:
            tmp_label.append(tag_map[label])
        fnl_labels.append(tmp_label)
    return fnl_labels

tr_label_list = preprocess_label(tag_map, tr_labels_list)
ts_label_list = preprocess_label(tag_map, ts_labels_list)
print(tr_label_list[0])
#print(tr_tokens_list[0])
#print(tr_data[0])
#print(tr_pos[0])
#print(tr_labels_list[0])
#print(params)

train_data = {'data':tr_data,'labels':tr_label_list[:5],'size':5}
val_data = {'data':ts_data,'labels':ts_label_list[:5],'size':5}

#import sys
#sys.exit()
# specify the train and val dataset sizes
params.train_size = train_data['size']
params.val_size = val_data['size']

logging.info("- done.")

# Define the model and optimizer
model = net.Net(params).cuda() if params.cuda else net.Net(params)
optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)
    
# fetch loss function and metrics
loss_fn = net.loss_fn
metrics = net.metrics

# Train the model
logging.info("Starting training for {} epoch(s)".format(params.num_epochs))
train_and_evaluate(model, train_data, val_data, optimizer, loss_fn, metrics, params, model_dir, data_dir,
                    restore_file)

Loading the datasets...
/home/adzuser/user_achyuta/venv3.6/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
- done.


[50, 45, 50, 8, 4, 20, 21, 45, 21, 8, 29]
[16]
[31, 21, 37, 18, 21, 4, 49, 21, 4, 49, 43, 31, 21, 18, 47, 21, 18, 47, 18, 21, 37, 37, 43, 18, 24, 21, 18, 21, 32, 49, 31, 21, 18, 31, 21, 37, 18, 31, 21, 29]
[31, 21, 18, 21, 37, 49, 21, 43, 18, 51, 24, 21, 18, 21, 37, 18, 21, 18, 21, 45, 50, 8, 5, 9, 37, 18, 47, 24, 21, 37, 10, 47, 31, 21, 18, 31, 21, 18, 31, 21, 18, 51, 31, 21, 32, 25, 49, 49, 48, 35, 21, 18, 31, 35, 18, 21, 18, 31, 21, 37, 29]
[57, 21, 37, 49, 24, 21, 18, 21, 4, 21, 18, 31, 32, 46, 23, 35, 18, 43, 18, 24, 21, 18, 21, 4, 10, 49, 21, 18, 11, 18, 21, 10, 15, 43, 35, 10, 43, 15, 4, 21, 32, 49, 18, 21, 18, 21, 10, 21, 37, 29]
[31, 21, 4, 20, 21, 45, 50, 8, 29]
[16]
[31, 21, 19, 21, 37, 19, 18, 21, 37, 18, 21, 18, 21, 49, 18, 47, 24, 20, 18, 11, 18, 31, 28, 21, 18, 57, 21, 18, 21, 18, 24, 21, 18, 20, 4, 49, 43, 37, 21, 37, 4, 37, 18, 21, 18, 21, 37, 29]
[21, 37, 49, 18, 31, 21, 31, 21, 37, 4, 32, 49, 31, 37, 21, 18, 31, 21, 18, 21, 37, 18, 47, 21, 18, 21, 10, 21, 4, 43, 51, 

Starting training for 10 epoch(s)
  0%|          | 0/1 [00:00<?, ?it/s]

------------------------------
(5,)
(5, 61, 100)
(5, 61)
50
(50, 100)
11
(11, 100)
61
(61, 100)
40
(40, 100)
1
(1, 100)
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.int64
torch.int64
tensor([[    0,     0,     0,  ...,    -1,     1,     0],
        [    0,     0,     0,  ...,     0,     0,     0],
        [    0,     0,     0,  ...,     0,     0,     0],
        ...,
        [19950, 19950, 19950,  ..., 19950, 19950, 19950],
        [19950, 19950, 19950,  ..., 19950, 19950, 19950],
        [19950, 19950, 19950,  ..., 19950, 19950, 19950]], device='cuda:0')
-------------------------
torch.Size([5, 61, 100])
torch.Size([5, 61])


RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'mat2'

In [10]:
params.embedding_dim

100

In [22]:
train_and_evaluate(model, train_data, val_data, optimizer, loss_fn, metrics, params, model_dir, data_dir,
                    restore_file)


  0%|          | 0/1 [00:00<?, ?it/s]

------------------------------
(5,)
(5, 61, 100)
(5, 61)
50
(50, 100)
11
(11, 100)
61
(61, 100)
40
(40, 100)
1
(1, 100)
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.int64
torch.int64
tensor([[    1,     0,     0,  ...,     0,     0,     1],
        [    0,     0,     0,  ...,     0,     0,     0],
        [    0,     0,     0,  ...,     0,     0,     0],
        ...,
        [19950, 19950, 19950,  ..., 19950, 19950, 19950],
        [19950, 19950, 19950,  ..., 19950, 19950, 19950],
        [19950, 19950, 19950,  ..., 19950, 19950, 19950]], device='cuda:0')
-------------------------
torch.Size([5, 61, 100])
torch.Size([5, 61])


RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'mat2'

In [11]:
model['australia'].dtype()

TypeError: 'Net' object is not subscriptable

In [4]:
train_data

{'data': [[0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   9,
   15,
   1,
   16,
   17,
   18,
   19,
   20,
   21],
  [22,
   1,
   23,
   24,
   11,
   9,
   25,
   26,
   9,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   35,
   13,
   35,
   40,
   9,
   41,
   21,
   35],
  [42, 4, 18, 9, 43, 1, 44, 7, 45, 46, 11, 47, 48, 21],
  [49, 50, 9, 51, 1, 52, 53, 54, 55, 56, 57, 58, 59, 60, 21],
  [61,
   8,
   62,
   63,
   9,
   64,
   1,
   9,
   65,
   66,
   1,
   67,
   68,
   69,
   70,
   71,
   11,
   9,
   72,
   73,
   74,
   75,
   1,
   76,
   21],
  [61,
   77,
   78,
   79,
   80,
   67,
   68,
   81,
   11,
   9,
   12,
   25,
   13,
   9,
   82,
   83,
   1,
   84,
   16,
   17,
   11,
   19,
   20,
   21],
  [61,
   6,
   85,
   86,
   87,
   1,
   88,
   89,
   90,
   11,
   91,
   92,
   93,
   94,
   95,
   93,
   96,
   93,
   13,
   97,
   21],
  [61,
   98,
   99,
   100,


# Evaluation

In [6]:
"""
    Evaluate the model on the test set.
"""

restore_file='best'

# Load the parameters
json_path = os.path.join(model_dir, 'params.json')
assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
params = utils.Params(json_path)

# use GPU if available
params.cuda = torch.cuda.is_available()     # use GPU is available

# Set the random seed for reproducible experiments
torch.manual_seed(230)
if params.cuda: torch.cuda.manual_seed(230)

# load data
data_loader = DataLoader(data_dir, params)
data = data_loader.load_data(['test'], data_dir)
test_data = data['test']

# specify the test set size
params.test_size = test_data['size']
test_data_iterator = data_loader.data_iterator(test_data, params)

# Define the model
model = net.Net(params).cuda() if params.cuda else net.Net(params)
    
loss_fn = net.loss_fn
metrics = net.metrics

# Reload weights from the saved file
utils.load_checkpoint(os.path.join(model_dir, restore_file + '.pth.tar'), model)

# Evaluate
num_steps = (params.test_size + 1) // params.batch_size
test_metrics = evaluate(model, loss_fn, test_data_iterator, metrics, params, num_steps)
save_path = os.path.join(model_dir, "metrics_test_{}.json".format(restore_file))
utils.save_dict_to_json(test_metrics, save_path)
print(test_metrics)

{'accuracy': 0.7427669902912621, 'loss': 1.8864147663116455}


In [3]:
!pip install nltk

    100% |████████████████████████████████| 1.4MB 790kB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/c5/10/369f50bcd4621b263927b0a1519987a04383d4a98fb10438042ad410cf88/singledispatch-3.4.0.3-py2.py3-none-any.whl
  Stored in directory: /home/achyuta/.cache/pip/wheels/4b/c8/24/b2343664bcceb7147efeb21c0b23703a05b23fcfeaceaa2a1e
Successfully built nltk
